# Chapter 10: Access Modifiers and Encapsulation

---

## 10.1 Understanding Encapsulation

Encapsulation is one of the four pillars of object-oriented programming (along with inheritance, polymorphism, and abstraction). It refers to the bundling of data and methods that operate on that data within a single unit (class), while restricting direct access to some of the object's components.

**The Concept of Encapsulation:**

```typescript
// ❌ Poor encapsulation - everything is public and exposed
class BadBankAccount {
  balance: number = 0; // Directly exposed
  
  constructor(initialBalance: number) {
    this.balance = initialBalance;
  }
}

const badAccount = new BadBankAccount(100);
badAccount.balance = -1000; // ❌ Direct modification allowed!
badAccount.balance = 999999; // ❌ No validation possible!

// ✅ Good encapsulation - data is protected
class GoodBankAccount {
  private _balance: number = 0; // Private - cannot be accessed directly
  
  constructor(initialBalance: number) {
    if (initialBalance < 0) {
      throw new Error("Initial balance cannot be negative");
    }
    this._balance = initialBalance;
  }
  
  // Controlled access through methods
  getBalance(): number {
    return this._balance;
  }
  
  deposit(amount: number): void {
    if (amount <= 0) {
      throw new Error("Deposit amount must be positive");
    }
    this._balance += amount;
  }
  
  withdraw(amount: number): boolean {
    if (amount <= 0) {
      throw new Error("Withdrawal amount must be positive");
    }
    if (amount > this._balance) {
      return false; // Insufficient funds
    }
    this._balance -= amount;
    return true;
  }
}

const goodAccount = new GoodBankAccount(100);
// goodAccount._balance = -1000; // ❌ Error: Property '_balance' is private
goodAccount.deposit(50);        // ✅ Controlled modification
console.log(goodAccount.getBalance()); // 150
```

**Why Encapsulation Matters:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Benefits of Encapsulation                           │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   1. Data Protection                                                │
│      • Prevents invalid states (negative balances, empty names)      │
│      • Ensures invariants are maintained                             │
│      • Controls how data is modified                                 │
│                                                                     │
│   2. Implementation Hiding                                            │
│      • Internal details can change without affecting consumers       │
│      • Reduces coupling between classes                              │
│      • Allows refactoring with minimal impact                        │
│                                                                     │
│   3. Validation & Logic                                               │
│      • All modifications go through validation logic                   │
│      • Side effects can be managed (logging, notifications)          │
│      • Complex calculations can be cached and updated atomically     │
│                                                                     │
│   4. Interface Stability                                              │
│      • Public API remains stable while internals evolve              │
│      • Clear contract between class and consumers                  │
│      • Easier to document and test                                   │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 10.2 Deep Dive into Access Modifiers

TypeScript provides three access modifiers that control the visibility of class members: `public`, `private`, and `protected`.

### 10.2.1 Public Members

`public` is the default accessibility modifier in TypeScript. Public members can be accessed from anywhere: inside the class, from subclasses, and from external code.

**Public Access Pattern:**

```typescript
class User {
  // Explicitly public (though public is default)
  public name: string;
  public email: string;
  
  constructor(name: string, email: string) {
    this.name = name;
    this.email = email;
  }
  
  // Public method
  public greet(): string {
    return `Hello, I'm ${this.name}`;
  }
  
  // Also public (default)
  updateEmail(newEmail: string): void {
    this.email = newEmail;
  }
}

// Usage - all members accessible
const user = new User("John", "john@example.com");
console.log(user.name);      // ✅ OK - public property
console.log(user.email);     // ✅ OK - public property
console.log(user.greet());   // ✅ OK - public method

user.name = "Jane";          // ✅ OK - can modify public properties
user.updateEmail("jane@example.com"); // ✅ OK
```

**When to Use Public:**

```typescript
// Public is appropriate for:
// 1. Configuration objects that need to be modified
class Config {
  public apiUrl: string;
  public timeout: number;
  
  constructor(apiUrl: string, timeout: number) {
    this.apiUrl = apiUrl;
    this.timeout = timeout;
  }
}

// 2. Data Transfer Objects (DTOs) - simple data containers
class UserDTO {
  constructor(
    public id: number,
    public name: string,
    public email: string
  ) {}
}

// 3. Public API surface that is meant to be used
class Calculator {
  public add(a: number, b: number): number {
    return a + b;
  }
  
  public subtract(a: number, b: number): number {
    return a - b;
  }
}
```

### 10.2.2 Private Members

`private` members can only be accessed from within the class that declares them. They are not accessible from subclasses or external code.

**Private Implementation Details:**

```typescript
class SecureAccount {
  // Private properties - internal state only
  private _balance: number;
  private _accountNumber: string;
  private _pinHash: string;
  private _transactionLog: string[] = [];
  
  constructor(accountNumber: string, initialBalance: number, pin: string) {
    this._accountNumber = accountNumber;
    this._balance = initialBalance;
    this._pinHash = this._hashPin(pin);
  }
  
  // Private method - internal implementation only
  private _hashPin(pin: string): string {
    // Simple hash for demonstration
    return `hash_${pin.split('').reverse().join('')}`;
  }
  
  // Private method
  private _validatePin(pin: string): boolean {
    return this._hashPin(pin) === this._pinHash;
  }
  
  // Private method
  private _logTransaction(type: string, amount: number): void {
    const timestamp = new Date().toISOString();
    this._transactionLog.push(`[${timestamp}] ${type}: $${amount}`);
  }
  
  // Public API
  public getBalance(): number {
    return this._balance;
  }
  
  public withdraw(amount: number, pin: string): boolean {
    if (!this._validatePin(pin)) {
      console.log("Invalid PIN");
      return false;
    }
    
    if (amount > this._balance) {
      console.log("Insufficient funds");
      return false;
    }
    
    this._balance -= amount;
    this._logTransaction("WITHDRAW", amount);
    return true;
  }
  
  public deposit(amount: number): void {
    if (amount <= 0) {
      throw new Error("Invalid amount");
    }
    this._balance += amount;
    this._logTransaction("DEPOSIT", amount);
  }
}

const account = new SecureAccount("123456", 1000, "1234");

// Public methods work
account.deposit(500);
console.log(account.getBalance()); // 1500

// Private members are inaccessible
// console.log(account._balance);        // ❌ Error: Property '_balance' is private
// console.log(account._accountNumber);  // ❌ Error: Property '_accountNumber' is private
// account._validatePin("1234");         // ❌ Error: Property '_validatePin' is private

// But we can access them indirectly through public methods
account.withdraw(200, "1234"); // Works if PIN is correct
```

**Private Member Naming Convention:**

```typescript
class Example {
  // Convention: prefix private members with underscore
  private _internalState: string;
  private _cachedValue: number;
  
  // Some teams use trailing underscore instead
  private internalState_: string;
  
  // Modern alternative: no prefix, rely on access modifier
  private internalState: string;
  
  constructor() {
    this._internalState = "initial";
    this.internalState = "initial";
  }
  
  // Getter without underscore in public API
  get state(): string {
    return this._internalState;
  }
}

// The underscore prefix is a convention that:
// 1. Makes it clear when reading code that it's private
// 2. Helps distinguish from public properties in getters/setters
// 3. Popular in languages like C#, Python, and JavaScript
```

### 10.2.3 Protected Members

`protected` members are accessible within the class that declares them and within subclasses (derived classes), but not from external code.

**Protected for Extension Points:**

```typescript
class DatabaseConnection {
  // Protected properties - available to subclasses
  protected connectionString: string;
  protected isConnected: boolean = false;
  protected retryCount: number = 0;
  
  constructor(connectionString: string) {
    this.connectionString = connectionString;
  }
  
  // Public API
  public connect(): void {
    if (!this.isConnected) {
      this.establishConnection();
      this.isConnected = true;
    }
  }
  
  public disconnect(): void {
    if (this.isConnected) {
      this.closeConnection();
      this.isConnected = false;
    }
  }
  
  // Protected methods - can be overridden by subclasses
  protected establishConnection(): void {
    console.log(`Connecting to ${this.connectionString}...`);
    // Base implementation
  }
  
  protected closeConnection(): void {
    console.log("Closing connection...");
  }
  
  // Protected helper
  protected logQuery(sql: string): void {
    console.log(`Executing: ${sql}`);
  }
}

class PostgresConnection extends DatabaseConnection {
  private sslMode: boolean;
  
  constructor(connectionString: string, sslMode: boolean) {
    super(connectionString);
    this.sslMode = sslMode;
  }
  
  // Can access protected members from parent
  public enableSSL(): void {
    // Accessing protected property from subclass
    if (this.isConnected) {
      console.log("Cannot change SSL while connected");
      return;
    }
    this.sslMode = true;
  }
  
  // Override protected method
  protected establishConnection(): void {
    // Accessing protected property
    console.log(`Establishing PostgreSQL connection to ${this.connectionString}`);
    if (this.sslMode) {
      console.log("Using SSL mode");
    }
    // Custom logic for PostgreSQL
    super.establishConnection(); // Can call parent implementation
  }
  
  public query(sql: string): void {
    // Can use protected method from parent
    this.logQuery(sql);
    // Execute query...
  }
}

const pg = new PostgresConnection("localhost:5432/mydb", true);
pg.connect(); // Uses overridden establishConnection
pg.query("SELECT * FROM users");

// Protected members not accessible externally
// pg.connectionString;      // ❌ Error: Property is protected
// pg.establishConnection(); // ❌ Error: Property is protected
// pg.isConnected;           // ❌ Error: Property is protected
```

**Protected Constructor Pattern:**

```typescript
// Protected constructor prevents direct instantiation but allows inheritance
abstract class Logger {
  protected constructor() {
    // Initialize logging infrastructure
  }
  
  abstract log(message: string): void;
}

// Can extend
class FileLogger extends Logger {
  constructor(private filePath: string) {
    super(); // Must call protected constructor
  }
  
  log(message: string): void {
    console.log(`Writing to ${this.filePath}: ${message}`);
  }
}

const fileLogger = new FileLogger("app.log"); // OK
// const logger = new Logger(); // ❌ Error: Constructor is protected
```

---

## 10.3 Parameter Properties (Shorthand)

TypeScript provides a convenient shorthand for declaring and initializing properties in constructors, reducing boilerplate code.

### 10.3.1 Constructor Parameter Properties

**Without Parameter Properties (Verbose):**

```typescript
class UserVerbose {
  private name: string;
  private age: number;
  private email: string;
  public id: number;
  
  constructor(name: string, age: number, email: string, id: number) {
    this.name = name;
    this.age = age;
    this.email = email;
    this.id = id;
  }
  
  getInfo(): string {
    return `${this.name} (${this.age}) - ${this.email}`;
  }
}
```

**With Parameter Properties (Concise):**

```typescript
class UserConcise {
  constructor(
    private name: string,
    private age: number,
    private email: string,
    public id: number
  ) {}
  
  getInfo(): string {
    return `${this.name} (${this.age}) - ${this.email}`;
  }
}

// Both are equivalent, but the second is much cleaner
const user = new UserConcise("John", 30, "john@example.com", 1);
console.log(user.id); // 1 (public)
// console.log(user.name); // ❌ Error: private
console.log(user.getInfo()); // "John (30) - john@example.com"
```

### 10.3.2 Combining with Modifiers

**All Combinations:**

```typescript
class Employee {
  constructor(
    // Accessible everywhere
    public name: string,
    
    // Accessible only in this class
    private salary: number,
    
    // Accessible in this class and subclasses
    protected department: string,
    
    // Cannot be reassigned after initialization
    readonly employeeId: string,
    
    // Combining modifiers
    private readonly ssn: string,
    protected readonly hireDate: Date
  ) {}
  
  getSalary(): number {
    // Can access private property
    return this.salary;
  }
  
  getSSN(): string {
    // Can access private readonly
    return this.ssn;
  }
}

class Manager extends Employee {
  constructor(
    name: string,
    salary: number,
    department: string,
    employeeId: string,
    ssn: string,
    hireDate: Date,
    private teamSize: number
  ) {
    super(name, salary, department, employeeId, ssn, hireDate);
  }
  
  getDepartment(): string {
    // Can access protected property
    return this.department;
  }
  
  getHireDate(): Date {
    // Can access protected readonly
    return this.hireDate;
  }
  
  getInfo(): string {
    return `${this.name} manages ${this.teamSize} people in ${this.department}`;
    // Can access public name
    // Cannot access private salary or ssn
  }
}

const manager = new Manager(
  "Alice",
  100000,
  "Engineering",
  "EMP001",
  "123-45-6789",
  new Date("2020-01-15"),
  5
);

console.log(manager.name);        // ✅ public
console.log(manager.employeeId);  // ✅ public readonly
// manager.employeeId = "NEW";    // ❌ Error: readonly
// console.log(manager.salary);   // ❌ Error: private
// console.log(manager.ssn);      // ❌ Error: private
```

**Default Values with Parameter Properties:**

```typescript
class Config {
  constructor(
    public apiUrl: string,
    public timeout: number = 5000,           // Default value
    public retries: number = 3,               // Default value
    public readonly environment: string = "development"
  ) {}
}

const config1 = new Config("https://api.example.com");
console.log(config1.timeout);    // 5000 (default)
console.log(config1.retries);    // 3 (default)

const config2 = new Config("https://api.prod.com", 10000, 5, "production");
console.log(config2.timeout);    // 10000 (provided)
```

---

## 10.4 Encapsulation Best Practices

### 10.4.1 Data Hiding Principles

**Principle of Least Privilege:**

```typescript
// Grant minimum necessary access
class SecureDocument {
  // Private - internal state only
  private content: string;
  private encryptionKey: string;
  private accessLog: string[] = [];
  
  // Protected - for subclasses to override behavior
  protected validateContent(content: string): boolean {
    return content.length > 0;
  }
  
  // Public - external interface
  constructor(content: string, encryptionKey: string) {
    if (!this.validateContent(content)) {
      throw new Error("Invalid content");
    }
    this.content = content;
    this.encryptionKey = encryptionKey;
  }
  
  // Public - read-only access to content
  public getContent(): string {
    this.logAccess("READ");
    return this.decrypt();
  }
  
  // Public - controlled modification
  public updateContent(newContent: string): void {
    if (!this.validateContent(newContent)) {
      throw new Error("Invalid content");
    }
    this.logAccess("WRITE");
    this.content = this.encrypt(newContent);
  }
  
  // Private - implementation details
  private encrypt(data: string): string {
    return `encrypted_${data}_${this.encryptionKey}`;
  }
  
  private decrypt(): string {
    // Parse encrypted format
    return this.content.replace(`encrypted_`, "").replace(`_${this.encryptionKey}`, "");
  }
  
  private logAccess(action: string): void {
    this.accessLog.push(`${new Date().toISOString()}: ${action}`);
  }
}
```

### 10.4.2 Controlled Access Patterns

**Getter and Setter Pattern:**

```typescript
class Temperature {
  // Private backing field
  private _celsius: number;
  
  constructor(celsius: number) {
    this._celsius = celsius;
  }
  
  // Getter - computed property
  get celsius(): number {
    return this._celsius;
  }
  
  // Setter with validation
  set celsius(value: number) {
    if (value < -273.15) {
      throw new Error("Temperature below absolute zero");
    }
    this._celsius = value;
  }
  
  // Computed getter (no setter - read-only)
  get fahrenheit(): number {
    return (this._celsius * 9 / 5) + 32;
  }
  
  // Setter that converts
  set fahrenheit(value: number) {
    this.celsius = (value - 32) * 5 / 9; // Uses setter with validation
  }
  
  // Read-only computed property
  get kelvin(): number {
    return this._celsius + 273.15;
  }
}

const temp = new Temperature(25);
console.log(temp.celsius);     // 25
console.log(temp.fahrenheit);  // 77 (computed)

temp.celsius = 30;             // Uses setter
temp.fahrenheit = 100;         // Converts and validates
// temp.kelvin = 300;          // ❌ Error: No setter, read-only

// temp.celsius = -300;        // ❌ Error: Validation fails
```

**Factory Pattern with Encapsulation:**

```typescript
class DatabaseConnection {
  // Private constructor - cannot instantiate directly
  private constructor(
    private connectionString: string,
    private poolSize: number
  ) {}
  
  // Static factory method controls creation
  static createConnection(config: {
    host: string;
    port: number;
    database: string;
    poolSize?: number;
  }): DatabaseConnection {
    const connString = `${config.host}:${config.port}/${config.database}`;
    const size = config.poolSize ?? 10;
    
    // Validation logic in one place
    if (size < 1 || size > 100) {
      throw new Error("Pool size must be between 1 and 100");
    }
    
    return new DatabaseConnection(connString, size);
  }
  
  // Public API
  public query(sql: string): void {
    console.log(`Executing on ${this.connectionString}: ${sql}`);
  }
  
  public getPoolSize(): number {
    return this.poolSize;
  }
}

// Usage
const db = DatabaseConnection.createConnection({
  host: "localhost",
  port: 5432,
  database: "myapp",
  poolSize: 20
});

// const bad = new DatabaseConnection("conn", 10); // ❌ Error: Constructor is private
```

---

## 10.5 JavaScript Private Fields (#)

TypeScript supports ECMAScript private fields (using the `#` prefix), which provide true runtime privacy as opposed to TypeScript's compile-time-only `private` modifier.

### 10.5.1 Private Field Syntax

**Hash-Based Private Fields:**

```typescript
class ModernUser {
  // ECMAScript private fields - truly private at runtime
  #id: string;
  #password: string;
  #secretKey: string;
  
  // Public fields
  name: string;
  email: string;
  
  constructor(id: string, name: string, email: string, password: string) {
    this.#id = id;
    this.name = name;
    this.email = email;
    this.#password = this.#hashPassword(password);
    this.#secretKey = this.#generateKey();
  }
  
  // Private method
  #hashPassword(password: string): string {
    return `hashed_${password}_secure`;
  }
  
  // Private method
  #generateKey(): string {
    return Math.random().toString(36).substring(2);
  }
  
  // Public API
  validatePassword(password: string): boolean {
    return this.#hashPassword(password) === this.#password;
  }
  
  getId(): string {
    return this.#id;
  }
  
  // Private method using other private fields
  #internalAudit(): void {
    console.log(`Auditing user ${this.#id} with key ${this.#secretKey}`);
  }
  
  public audit(): void {
    this.#internalAudit();
  }
}

const user = new ModernUser("123", "John", "john@example.com", "secret123");

console.log(user.name);           // ✅ "John"
console.log(user.getId());        // ✅ "123"
user.audit();                     // ✅ Calls private method internally

// Runtime privacy - these don't exist outside the class
// console.log(user.#id);          // ❌ SyntaxError: Private field must be declared in enclosing class
// user.#hashPassword("test");     // ❌ SyntaxError
// user.#secretKey = "hacked";     // ❌ SyntaxError

// TypeScript private (compile-time only) vs ES private (runtime)
class Comparison {
  private tsPrivate: string = "TS";
  #jsPrivate: string = "JS";
}

const comp = new Comparison();
// (comp as any).tsPrivate; // Works at runtime! (TypeScript only checks compile time)
// (comp as any).#jsPrivate; // SyntaxError even with type assertion!
```

### 10.5.2 TypeScript's Private vs JS Private

**Detailed Comparison:**

```typescript
class HybridExample {
  // TypeScript private - compile-time check only
  private tsPrivate: string = "TypeScript Private";
  
  // ECMAScript private - runtime check
  #esPrivate: string = "ECMAScript Private";
  
  // TypeScript protected
  protected tsProtected: string = "TypeScript Protected";
  
  getValues(): object {
    return {
      ts: this.tsPrivate,
      es: this.#esPrivate
    };
  }
}

const hybrid = new HybridExample();

// TypeScript private bypass (works at runtime despite TypeScript error)
// This is why # is more secure for sensitive data
const bypassed = (hybrid as any).tsPrivate; // Works! No error at runtime
console.log(bypassed); // "TypeScript Private"

// ES private cannot be bypassed
// const cannotBypass = (hybrid as any).#esPrivate; // SyntaxError: Private field must be declared in enclosing class

// Even Object.keys doesn't reveal private fields
console.log(Object.keys(hybrid)); // [] - both are hidden from enumeration

// But ES private is truly encapsulated
// const descriptors = Object.getOwnPropertyDescriptors(hybrid);
// console.log(descriptors); // Won't show #esPrivate
```

**When to Use Each:**

```
┌─────────────────────────────────────────────────────────────────────┐
│            TypeScript private vs ECMAScript Private (#)              │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Use TypeScript 'private' when:                                    │
│   ✅ You need compatibility with ES5/ES3 targets                     │
│   ✅ Working with existing codebase using _ prefix convention      │
│   ✅ You need to support older browsers without transpilation        │
│   ✅ Performance is critical (# has slight overhead)                 │
│   ✅ You need to access private members in unit tests (with hacks) │
│                                                                     │
│   Use ECMAScript '#' when:                                          │
│   ✅ You need true runtime privacy (security-critical code)          │
│   ✅ You want to prevent any external access, even with hacks        │
│   ✅ Working with modern ES2022+ environments only                  │
│   ✅ Building libraries where encapsulation is crucial              │
│   ✅ You want hard guarantees, not just compile-time checks         │
│                                                                     │
│   Recommendation:                                                   │
│   • Use # for truly sensitive data (passwords, tokens, keys)         │
│   • Use private for internal implementation details                 │
│   • Gradually migrate to # as ES2022 becomes standard                │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

**Private Fields in Inheritance:**

```typescript
class Parent {
  #parentSecret: string = "parent";
  protected familySecret: string = "family";
  
  revealParent(): string {
    return this.#parentSecret;
  }
}

class Child extends Parent {
  #childSecret: string = "child";
  
  // Cannot access parent's #parentSecret
  revealAll(): string {
    // return this.#parentSecret; // ❌ Error: Property '#parentSecret' is not accessible
    return `Child: ${this.#childSecret}, Family: ${this.familySecret}`;
  }
}

const child = new Child();
console.log(child.revealParent()); // "parent" - accessible via public method
console.log(child.revealAll());    // "Child: child, Family: family"

// Each class has its own private namespace
// Parent.#parentSecret !== Child.#parentSecret (they're completely separate)
```

---

## 10.6 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we explored access modifiers and encapsulation in depth:

**Key Takeaways:**

1. **Encapsulation**:
   - Bundles data and methods together
   - Hides implementation details
   - Exposes controlled public interfaces
   - Prevents invalid object states

2. **Access Modifiers**:
   - `public` (default): Accessible everywhere
   - `private`: Accessible only within declaring class
   - `protected`: Accessible within class and subclasses

3. **Parameter Properties**:
   - Shorthand for declaring and initializing in constructor
   - Combines parameter with property declaration
   - Supports all modifiers: public, private, protected, readonly

4. **Best Practices**:
   - Default to private, expose only what's necessary
   - Use getters/setters for controlled access
   - Validate all state changes
   - Protect invariants through encapsulation

5. **Private Fields**:
   - TypeScript `private`: Compile-time only checking
   - ECMAScript `#`: True runtime privacy
   - Use `#` for security-sensitive data
   - Private fields are not inherited (each class has its own)

### Practical Exercises

**Exercise 1: Access Modifier Refactoring**

Refactor this poorly encapsulated code:

```typescript
// Refactor this:
class BankAccountBad {
  balance: number;
  accountNumber: string;
  transactions: string[] = [];
  
  constructor(balance: number, accountNumber: string) {
    this.balance = balance;
    this.accountNumber = accountNumber;
  }
}

// To use:
// - Private balance (prevent negative)
// - Private accountNumber (expose last 4 digits only)
// - Protected method for logging transactions
// - Public methods for deposit/withdraw with validation
// - Readonly account creation date
```

**Exercise 2: Parameter Properties**

Convert this verbose class to use parameter properties:

```typescript
class Employee {
  private id: number;
  private name: string;
  protected department: string;
  public readonly hireDate: Date;
  private salary: number;
  
  constructor(id: number, name: string, department: string, hireDate: Date, salary: number) {
    this.id = id;
    this.name = name;
    this.department = department;
    this.hireDate = hireDate;
    this.salary = salary;
  }
}
```

**Exercise 3: Encapsulation Design**

Design a secure messaging system:

```typescript
// Create a Message class with:
// - Private content (encrypted)
// - Private sender/recipient
// - Public timestamp (readonly)
// - Protected method for encryption/decryption
// - Public method to read content (decrypts on the fly)
// - Public method to reply (creates new Message)
// - Use ES private fields (#) for encryption keys
// - Ensure content cannot be modified after creation
```

**Exercise 4: Factory Pattern with Private Constructors**

Implement a connection pool:

```typescript
// Create a DatabaseConnection class with:
// - Private constructor
// - Static factory method createConnection(config)
// - Private connection state
// - Public query method
// - Protected reconnect method for subclasses
// - Static method getPool() that manages instances (singleton pattern)
```

**Exercise 5: Mixed Access Levels**

Build a user management system:

```typescript
// Create a User class with:
// - Public: username, profile picture URL
// - Protected: role, permissions array
// - Private: password hash, email, security questions
// - Readonly: userId, registrationDate
// - Use parameter properties in constructor
// - Create Admin class that extends User
// - Admin can view (but not modify) other users' roles
// - Admin cannot see private user data (email, passwords)
// - Implement secure password verification
```

### Additional Resources

- **TypeScript Handbook - Classes**: https://www.typescriptlang.org/docs/handbook/2/classes.html
- **MDN - Private Class Fields**: https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Classes/Private_class_fields
- **TypeScript Private vs Protected**: https://www.typescriptlang.org/docs/handbook/2/classes.html#private
- **Encapsulation in OOP**: https://en.wikipedia.org/wiki/Encapsulation_(computer_programming)

---

## Coming Up Next: Chapter 11 - Union and Intersection Types

In the next chapter, we will explore:

- Union types (`string | number`) for flexible typing
- Type narrowing with `typeof`, `instanceof`, and type guards
- Discriminated unions for type-safe state management
- Intersection types (`Person & Serializable`) for combining types
- Working with nullable types
- Exhaustiveness checking with the `never` type

Union and intersection types are fundamental TypeScript features that allow you to express complex type relationships and create flexible, type-safe APIs.

---

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='9. inheritance_and_polymorphism.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../4. advanced_types/11. union_and_intersection_types.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
